In [2]:
# # 필요한 라이브러리
# !pip install python-dotenv
# !pip install speechrecognition
# !pip install openai
# !pip install pyttsx3


In [3]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
from openai import OpenAI

def make_theme(query, temperature=0.3):
    client = OpenAI()

    system_instruction = '''
    당신은 누구보다 뛰어난 상담가 입니다.
    연애, 진로, 가족, 친구 등 다양한 고민에 대해 공감하고 조언할 수 있습니다.


    ### 지시사항 ###
    - 사용자의 감정을 이해하고 공감 먼저 할것
    - 적절한 질문을 통해 사용자의 고민을 깊이 있게 탐색할 것
    - 사용자가 스스로 해결책을 찾을 수 있도록 유도할 것
    - 현실적인 조언을 제공할 것

    ### 출력 형식 ###
    사용자: 애인이랑 자주 싸워서 힘들어
    봇: 힘들었구나.. 어떤 일이 있었는지 더 이야기해줄 수 있어?
    사용자: 별것도 아닌 일로 자주 싸워
    봇: 자주 싸우는 일이 정말 스트레스겠구나. 어떤 일로 주로 싸우게 되는지 말해줄래?
    사용자: (사용자의 고민)
    봇: (사용자의 감정을 공감하고, 고민을 더 깊이 탐색하는 질문이나 조언)
    '''
    user_message = f'''
    {query}
    '''

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[
            {'role':'system','content':system_instruction},
            {'role':'user','content':query}
        ],
        temperature=temperature,
        max_tokens=2048
    )
    
    return response.choices[0].message.content

In [5]:
import pyttsx3
import time

def read_chatbot(chatbot_message):
    engine = pyttsx3.init()
    engine.setProperty('rate', 200)  # 말하기 속도 조절
    engine.say(chatbot_message)
    
    # 화면 출력
    print('🤖: ', end='')
    for sentence in chatbot_message.split('.'):
        print(sentence.strip())
        time.sleep(0.2)
    
    engine.runAndWait()
    print()


In [6]:
import speech_recognition as sr

def listen_chatbot():
    recognizer = sr.Recognizer()
    
    while True:
        with sr.Microphone() as source:
            print('👤: ', end='')
            audio = recognizer.listen(source)
            try:
                command = recognizer.recognize_google(audio, language='ko')
                if command:
                    print(command, '\n')
                    break
            except sr.UnknownValueError:
                print("⚠️ 음성을 인식하지 못했습니다. 다시 말해주세요.")
            except sr.RequestError as e:
                print(f"⚠️ Google Speech API 오류: {e}")
                break
    
    return command


In [ ]:
def run_voice_chatbot():
    print("🎙️ 대화 시작 ('그만'이라고 말하면 종료됩니다)\n")

    while True:
        user_input = listen_chatbot()

        if user_input.strip() in ["그만"]:
            read_chatbot("대화를 종료합니다. 안녕!")
            break
        
        try:
            bot_reply = make_theme(user_input)
        except Exception as e:
            print(f"⚠️ 챗봇 응답 오류: {e}")
            bot_reply = "죄송해요, 지금은 답변을 할 수 없습니다."
        
        read_chatbot(bot_reply)


In [13]:
from openai import chat


def run_voice_chatbot():
    print("🎙️ 대화 시작 ('그만'이라고 말하면 종료됩니다)\n")
    chat_history = []
    while True:
        user_input = listen_chatbot()

        if user_input.strip() in ["그만"]:
            read_chatbot("대화를 종료합니다. 안녕!")
            break
        
        # 사용자 메시지를 대화 기록에 추가
        chat_history.append({"role": "user", "content": user_input})

        try:
            # GPT 호출 시 대화 기록 전체를 보내기
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=chat_history,
                temperature=0.3,
                max_tokens=2048
            )
            bot_reply = response.choices[0].message.content

            # 챗봇 답변도 기록에 추가
            chat_history.append({"role": "assistant", "content": bot_reply})
        
        except Exception as e:
            print(f"⚠️ 챗봇 응답 오류: {e}")
            bot_reply = "죄송해요, 지금은 답변을 할 수 없습니다."
            chat_history.append({"role": "assistant", "content": bot_reply})
        
        read_chatbot(bot_reply)

In [14]:
run_voice_chatbot()


🎙️ 대화 시작 ('그만'이라고 말하면 종료됩니다)

👤: 내 이름은 김수연 

🤖: 안녕하세요, 김수연님! 만나서 반갑습니다
어떻게 도와드릴까요?

👤: 내 이름이 뭐라고 

🤖: 당신의 이름은 김수연이라고 하셨습니다
맞나요?

👤: ⚠️ 음성을 인식하지 못했습니다. 다시 말해주세요.
👤: 그만 

🤖: 대화를 종료합니다
안녕!

